In [5]:
import matplotlib.pyplot as plt  
import numpy as np
from sklearn import datasets,decomposition,manifold
from andylearn import mnist, KNNneighbor
from mpl_toolkits.mplot3d import Axes3D
#%matplotlib inline

In [6]:
def LLE(data, k, d):
    '''
    data is the original data. m*n(m=samples, n=dimensions)
    '''
    data = np.matrix(data)
   
    N = data.shape[0]
    D = data.shape[1]
    # step 1
    # find the nearest neighbors (k). Here we get the distance from (x-y)^2
    # distance
    data_tmp = np.sum(np.multiply(data, data), axis=1)
    distance=np.mat(data_tmp + data_tmp.T - 2*data*data.T)
    
    sort_index = np.argsort(distance,axis=1)

    # k nearest neibors
    neibors = sort_index[:,1:k+1]

    # step 2
    # weights
    
    W = np.zeros((N, N))
    
    # Gram Matrix
    if(k > D):
        add = 0.001
    else:
        add = 0
    for i in range(N):
        neibors_i = neibors[i].A[0]
        G = data[neibors_i[:]] - data[i] 
        G = np.dot(G, G.T)
        G = G + add * np.dot(np.eye(k), np.trace(G))
        G_I = np.linalg.inv(G)
        w_i = np.sum(G_I,axis=1) / np.sum(G_I)
        w_i = w_i.flatten([0])    
        W[i][neibors[i]]=w_i
    
    # Step 3
    # M 
    
    M=np.eye(N) - W
    M=np.dot(M.T, M)
    
    eig_values, eig_vectors = np.linalg.eig(M)
    
    eig_index = np.argsort(eig_values)
    eig_index = eig_index[1:d+1]
    output = np.mat(eig_vectors[:,eig_index])
    print("LLE done")
    return output

In [8]:
def LLE_MNIST_2D():
    
    data, labels = mnist.load_mnist_data("./data/mnist/train-images", "./data/mnist/train-labels", 2000)
    low_data= LLE(data, 8, 2)

    print("drawing")
    fig = plt.figure(figsize=(10,7))
    plotwindow = fig.add_subplot(111)
    a = np.array(low_data)

    # draw the scatters
    color = ['green','dodgerblue','chartreuse','turquoise','orange','coral','salmon','darkgray','darkred','mediumpurple']
    marker = ["$0$","$1$","$2$","$3$","$4$","$5$","$6$","$7$","$8$","$9$"]
    for j in range(len(low_data)):
        plt.scatter(a[j][0], a[j][1],s=40, c=color[labels[j]],marker=marker[labels[j]])
    plt.show()
#LLE_MNIST_2D()

In [11]:
def LLE_MNIST_Acuracy():
    data, labels = mnist.load_mnist_data("./data/mnist/train-images", "./data/mnist/train-labels", 2000)
    low_data= LLE(data, 8, 10)
    s=neighbor.accuracy(low_data[0:1000], labels[0:1000], low_data[1000:2000], labels[1000:2000], 1)
    print(s)
LLE_MNIST_Acuracy()

data loaded
LLE done
0.824
